### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2017 ###

# Optimizing Class Scheduling #

#### Marcus Millin (marcus.millin@wisc.edu) and Ahad Zaman ( ahad.zaman@wisc.edu)

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
  1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

<!-- The first few sentences should give a quick overview of the entire project. Then, elaborate with a description of the problem that will be solved, a brief history (with [citations](https://en.wikipedia.org/wiki/Citation)) of how the problem came about, why it's important/interesting, and any other interesting facts you'd like to talk about. You should address and explain where the problem data is coming from (research? the internet? synthetically generated?) Also give an outline of the rest of the report.

This section should be 300-600 words long, and **should be accessible to a general audience** (don't assume your reader has taken the class!). Feel free to include images if you think it'll be helpful:

![fixit flowchart][flow]

For more help on using Markdown, see [this reference](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet).

[flow]: https://s-media-cache-ak0.pinimg.com/736x/f5/75/c5/f575c53b93724808c6f0211890a54900.jpg -->

When scheduling classes for a semester at UW-Madison, a lot of options are presented; even after having selected which courses you would like to take, you still need to find a combination of discussion sections, lectures, and labs which both is possible to do, and works well for your extra cirricular schedule. The first part of this problem is solved by the UW-Madison Schedule Planner, which will take your selected courses and generate a list of schedules which have all of the courses you selected without any overlap (if possible). However, due to the wide variety of times for discussion sections and lectures for a course, and only needing to take one of them a large number of potential schedules may be available. This can result in hundreds of possible schedules with no way to easily sort through them or determine which is the best for you.

While "which is best" varies from person to person, there are a few things that are often targeted by students when looking for a schedule. One of the things that makes one schedule better than another is the starting time of the first class and the ending time of the last class. Some people like to group their classes all in the morning to get them out of the way, and have their afternoons as free as possible, while others like to sleep in and would rather their day start later and might care less about when their final class gets done (within reason!).

With our model, we attempt to optimize class selection, keeping these in mind to find the "best" possible schedule for a selection of courses. We will be using data from the UW-Madison course guide for the Fall 2017 semester. In order to generate a large number of possible schedules, we have selected five classes with lots of options for discussion sections and lectures, representing a possible freshmen-level courseload. The classes are Comp Sci 240, Math 320, Philosophy 101, and Philosophy 210. Using the UW-Madison scheduling tool, this gives a total of over 1000 viable schedules assuming every class had seats available.

## 2. Mathematical model ##

<!-- A discussion of the modeling assumptions made in the problem (e.g. is it from physics? economics? something else?). Explain the decision variables, the constraints, and the objective function. Finally, show the optimization problem written in standard form. Discuss the model type (LP, QP, MIP, etc.). Equations should be formatted in $\LaTeX$ within the IJulia notebook. For this section you may **assume the reader is familiar with the material covered in class**. -->

First, in order to get a better grasp of the problem, we have listed the constraints and goals of this problem in plain english.
#### Constraints
* If a discussion is taken, its corresponding lecture must be taken.
* Take exactly one discussion for each course.
    * Implicit: Take all courses.
* Classes taken must not overlap.

####  Goals
1. Finish days as early as possible to give free time in the afternoon.
2. Start days as late as possible to give time to sleep in.
### Model
We constructed a Mixed Integer Program with 74 variables to solve this problem.
Assuming $x_i$ is a binary variable representing whether you are taking class i, $st_i$ is the start time of class i, and $et_i$ represents the end time of class i.

### For late starts

$\underset{x}{\text{maximize}} \qquad \sum_{i, j}{x_{ij}st_{ij}}$

$\begin{aligned}
\text{subject to} \qquad& x_i + x_j \le 1 &\forall i,j \text{ classes that overlap} \\
& \sum_{i \in \text{j's discussions}} x_i \le x_j &\forall j \text{ lectures} \\
& \sum_{i \in \text{course j}} x_i = 2 &\forall j \text{ courses} \\
& \sum_{i \in \text{j's lectures}} x_i \le 1 &\forall j \text{ courses}
\end{aligned}$

### For early ends
$\underset{x}{\text{maximize}} \qquad \sum_{i, j}{-x_{ij}et_{ij}}$

$\begin{aligned}
\text{subject to} \qquad& x_i + x_j \le 1 &\forall i,j \text{ classes that overlap} \\
& \sum_{i \in \text{j's discussions}} x_i \le x_j &\forall j \text{ lectures} \\
& \sum_{i \in \text{course j}} x_i = 2 &\forall j \text{ courses} \\
& \sum_{i \in \text{j's lectures}} x_i \le 1 &\forall j \text{ courses}
\end{aligned}$

## 3. Solution ##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **I will be running your code**. I suggest having multiple code blocks separated by text blocks that explain the various parts of your solution. You may also solve several versions of your problem with different models/assumptions.

It's fine to call external packages such as `Gurobi`, but try to minimize the use of exotic libraries.

In [38]:
cs240raw = readcsv("CS240.csv");
math320raw = readcsv("MATH320.csv");
phil101raw = readcsv("PHILOS101.csv");
phil210raw = readcsv("PHILOS210.csv");

In [39]:
c240t = map(x -> (x == "") ? DateTime(0) : DateTime(x, "HH:MM"), cs240raw[:, 8:end]);
m320t = map(x -> (x == "") ? DateTime(0) : DateTime(x, "HH:MM"), math320raw[:, 8:end]);
p101t = map(x -> (x == "") ? DateTime(0) : DateTime(x, "HH:MM"), phil101raw[:, 8:end]);
p210t = map(x -> (x == "") ? DateTime(0) : DateTime(x, "HH:MM"), phil210raw[:, 8:end]);

In [40]:
function findCollisions(arg1, arg2)
    clashes = []
    for d in 1:2:10 # Day
        for i in 1:size(arg1, 1)
            for j in 1:size(arg2, 1)
                if arg1[i, d] != DateTime(0) &&
                    arg2[j, d] != DateTime(0) &&
                    ((arg1[i, d] - arg2[j, d + 1]).value <= 0) &&
                    ((arg1[i, d + 1] - arg2[j, d]).value >= 0)
                    push!(clashes, [i j])
                end
            end
        end
    end
    vcat(clashes...)
end

findCollisions (generic function with 1 method)

With the above and below code we are able to precompute all of the potential collisions between two courses with all of their classes. With this information, we can set up our constraints so that we only produce valid schedules.

In [41]:
# c240 & m320
c2m3 = findCollisions(c240t, m320t)
# c240 & p101
c2p1 = findCollisions(c240t, p101t)
# c240 & p210
c2p2 = findCollisions(c240t, p210t)
# m320 & p101
m3p1 = findCollisions(m320t, p101t)
# m320 & p210
m3p2 = findCollisions(m320t, p210t)
# p101 & p210
p1p2 = findCollisions(p101t, p210t);

In [42]:
function getDiscIndex(csvIndex, numDiscsPer)
    lectureNum = 0
    discNum = 0
    lectureNum = find(x->(x > csvIndex), 1:(numDiscsPer + 1):(csvIndex + numDiscsPer +1))[1] - 1
    
    discNum = csvIndex - (lectureNum - 1)*(numDiscsPer +  1) - 1
    [lectureNum discNum]
end

getDiscIndex (generic function with 1 method)

In [43]:
function getEndTimes(class, LecCsvIndex, DiscCsvIndex)
    if DiscCsvIndex == 0
        println("DiscCsvIndex was 0. Please only call this for discussions")
    end
    MonEndTime
    
    TueEndTime
    WedEndTime
    ThuEndTime
    FriEndTime
end

getEndTimes (generic function with 1 method)

In [44]:
using JuMP, Gurobi
m = Model(solver=GurobiSolver())

@variable(m, cs240[1:16], Bin)  # 2 Lectures, 7 discussions each
@variable(m, math320[1:28], Bin) # 4 Lectuers, 6 discussions each
@variable(m, phil101[1:20], Bin) # 4 Lectures, 4 discussions each
@variable(m, phil210[1:10], Bin) # 2 Lectures, 4 discussions each

# Must take a discussion and a lecture
@constraint(m, sum(cs240) == 2)
@constraint(m, sum(math320) == 2)
@constraint(m, sum(phil101) == 2) 
@constraint(m, sum(phil210) == 2)

# If you take a discussion, you must take its corresponding lecture
A = 1:(7 + 1):(16)
@constraint(m, sum(cs240[A])==1)
for i in 1:size(A, 1)
    @constraint(m, sum(cs240[(A[i] + 1):(A[i] + 7)]) <= cs240[A[i]])
end
A = 1:(6 + 1):(28)
@constraint(m, sum(math320[A])==1)
for i in 1:size(A, 1)
    @constraint(m, sum(math320[(A[i] + 1):(A[i] + 6)]) <= math320[A[i]])
end
A = 1:(4 + 1):(20)
@constraint(m, sum(phil101[A])==1)
for i in 1:size(A, 1)
    @constraint(m, sum(phil101[(A[i] + 1):(A[i] + 4)]) <= phil101[A[i]])
end
A = 1:(4 + 1):(10)
@constraint(m, sum(phil210[A])==1)
for i in 1:size(A, 1)
    @constraint(m, sum(phil210[(A[i] + 1):(A[i] + 4)]) <= phil210[A[i]])
end

# Overlapping classes constraints
# c240 & m320
for i in 1:size(c2m3, 1)
    c1 = c2m3[i, 1]
    c2 = c2m3[i, 2]
    @constraint(m, cs240[c1] + math320[c2] <= 1)
end
# c240 & p101
for i in 1:size(c2p1, 1)
    c1 = c2p1[i, 1]
    c2 = c2p1[i, 2]
    @constraint(m, cs240[c1] + phil101[c2] <= 1)
end
# c240 & p210
for i in 1:size(c2p2, 1)
    c1 = c2p2[i, 1]
    c2 = c2p2[i, 2]
    @constraint(m, cs240[c1] + phil210[c2] <= 1)
end
# m320 & p101
for i in 1:size(m3p1, 1)
    c1 = m3p1[i, 1]
    c2 = m3p1[i, 2]
    @constraint(m, math320[c1] + phil101[c2] <= 1)
end
# m320 & p210
for i in 1:size(m3p2, 1)
    c1 = m3p2[i, 1]
    c2 = m3p2[i, 2]
    @constraint(m, math320[c1] + phil210[c2] <= 1)
end
# p101 & p210
for i in 1:size(p1p2, 1)
    c1 = p1p2[i, 1]
    c2 = p1p2[i, 2]
    @constraint(m, phil101[c1] + phil210[c2] <= 1)
end

@objective(m, Max, 1)

solve(m)

Optimize a model with 174 rows, 74 columns and 468 nonzeros
Variable types: 0 continuous, 74 integer (74 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective -0

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: -0 
Pool objective bound 0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%


:Optimal

## getDiscIndex(8,6)

In [45]:
cscsv = find(x->(x > 0), getvalue(cs240))
mathcsv = find(x->(x > 0), getvalue(math320))
phil1csv = find(x->(x > 0), getvalue(phil101))
phil2csv = find(x->(x > 0), getvalue(phil210))
cs = getDiscIndex(cscsv[2], 7)
math = getDiscIndex(mathcsv[2], 6)
phil1 = getDiscIndex(phil1csv[2], 4)
phil2 = getDiscIndex(phil2csv[2], 4)
println("Take CS 240 Lecture #", cs[1], " Disc #", cs[2] )
println("Take Math 320 Lecture #", math[1], " Disc #", math[2] )
println("Take Philos 101 Lecture #", phil1[1], " Disc #", phil1[2] )
println("Take Philos 210 Lecture #", phil2[1], " Disc #", phil2[2] )

Take CS 240 Lecture #1 Disc #4
Take Math 320 Lecture #4 Disc #3
Take Philos 101 Lecture #1 Disc #1
Take Philos 210 Lecture #2 Disc #1


In [59]:
println("BASIC SCHEDULE")
println()
println("CS 240")
println()
println("    Lecture Times:")
c240lst = Dates.format(c240t[cscsv[1], 1:2:end], "HH:MM")
c240let = Dates.format(c240t[cscsv[1], 2:2:end], "HH:MM")
c240dst = Dates.format(c240t[cscsv[2], 1:2:end], "HH:MM")
c240det = Dates.format(c240t[cscsv[2], 2:2:end], "HH:MM")
println()
println("\t\tMon\tTue\tWed\tThu\tFri")
println()
println("\tStart:\t",join(c240lst, "\t"))
println("\tEnd:\t", join(c240let, "\t"))
println()
println("    Discussion Times:")
println()
println("\t\tMon\tTue\tWed\tThu\tFri")
println()
println("\tStart:\t", join(c240dst, "\t"))
println("\tEnd:\t", join(c240det, "\t"))
println()
println("Math 320")
println()
println("    Lecture Times:")
m320lst = Dates.format(m320t[mathcsv[1], 1:2:end], "HH:MM")
m320let = Dates.format(m320t[mathcsv[1], 2:2:end], "HH:MM")
m320dst = Dates.format(m320t[mathcsv[2], 1:2:end], "HH:MM")
m320det = Dates.format(m320t[mathcsv[2], 2:2:end], "HH:MM")
println()
println("\t\tMon\tTue\tWed\tThu\tFri")
println()
println("\tStart:\t", join(m320lst, "\t"))
println("\tEnd:\t", join(m320let, "\t"))
println()
println("    Discussion Times:")
println()
println("\t\tMon\tTue\tWed\tThu\tFri")
println()
println("\tStart:\t", join(m320dst, "\t"))
println("\tEnd:\t", join(m320det, "\t"))
println()
println("Philos 101")
println()
println("    Lecture Times:")
p101lst = Dates.format(p101t[phil1csv[1], 1:2:end], "HH:MM")
p101let = Dates.format(p101t[phil1csv[1], 2:2:end], "HH:MM")
p101dst = Dates.format(p101t[phil1csv[2], 1:2:end], "HH:MM")
p101det = Dates.format(p101t[phil1csv[2], 2:2:end], "HH:MM")
println()
println("\t\tMon\tTue\tWed\tThu\tFri")
println()
println("\tStart:\t", join(p101lst, "\t"))
println("\tEnd:\t", join(p101let, "\t"))
println()
println("    Discussion Times:")
println()
println("\t\tMon\tTue\tWed\tThu\tFri")
println()
println("\tStart:\t", join(p101dst, "\t"))
println("\tEnd:\t", join(p101det, "\t"))
println()
println("Philos 210")
p210lst = Dates.format(p210t[phil2csv[1], 1:2:end], "HH:MM")
p210let = Dates.format(p210t[phil2csv[1], 2:2:end], "HH:MM")
p210dst = Dates.format(p210t[phil2csv[2], 1:2:end], "HH:MM")
p210det = Dates.format(p210t[phil2csv[2], 2:2:end], "HH:MM")
println()
println("    Lecture Times:")
println()
println("\t\tMon\tTue\tWed\tThu\tFri")
println()
println("\tStart:\t", join(p210lst, "\t"))
println("\tEnd:\t", join(p210let, "\t"))
println()
println("    Discussion Times:")
println()
println("\t\tMon\tTue\tWed\tThu\tFri")
println()
println("\tStart:\t", join(p210dst, "\t"))
println("\tEnd:\t", join(p210det, "\t"))

BASIC SCHEDULE

CS 240

    Lecture Times:

		Mon	Tue	Wed	Thu	Fri

	Start:	11:00	00:00	11:00	00:00	11:00
	End:	11:50	00:00	11:50	00:00	11:50

    Discussion Times:

		Mon	Tue	Wed	Thu	Fri

	Start:	00:00	00:00	00:00	09:55	00:00
	End:	00:00	00:00	00:00	10:45	00:00

Math 320

    Lecture Times:

		Mon	Tue	Wed	Thu	Fri

	Start:	13:20	00:00	13:20	00:00	13:20
	End:	14:10	00:00	14:10	00:00	14:10

    Discussion Times:

		Mon	Tue	Wed	Thu	Fri

	Start:	00:00	09:55	00:00	00:00	00:00
	End:	00:00	10:45	00:00	00:00	00:00

Philos 101

    Lecture Times:

		Mon	Tue	Wed	Thu	Fri

	Start:	09:55	00:00	09:55	00:00	09:55
	End:	10:45	00:00	10:45	00:00	10:45

    Discussion Times:

		Mon	Tue	Wed	Thu	Fri

	Start:	00:00	11:00	00:00	00:00	00:00
	End:	00:00	11:50	00:00	00:00	00:00

Philos 210

    Lecture Times:

		Mon	Tue	Wed	Thu	Fri

	Start:	00:00	08:00	00:00	08:00	00:00
	End:	00:00	09:15	00:00	09:15	00:00

    Discussion Times:

		Mon	Tue	Wed	Thu	Fri

	Start:	00:00	12:05	00:00	00:00	00:00
	End:	00:00	12:55	00:00

Remember to make sure your code compiles! I will be running your code!

## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |\$1600 |
| col 2 is      | centered      |  \$12 |
| zebra stripes | are neat      |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

Our first result that we printed (BASIC SCHEDULE) was a standard optimized schedule that does the purpose of the current UW Schedule Planner. It avoids conflicts with other classes and makes sure that all classes one signs up for are in the schedule, including one lecture and one discussion. 
Below is a readable version:
<div style="width:image width px; font-size:80%; text-align:center;"><img title="Ion thruster" src="Basic.png" alt="Missing Image" width="width" height="height" style="padding-bottom:0.5em;" />BASIC SCHEDULE</div>

After optimizing our schedule for a student who would like to finish their classes as early as possible we get the schedule below.

After optimizing our schedule for a student who would like to start their classes as late as possible we get the schedule below.


## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.

Our findings gave us viable schedule options for students who want to specialize their schedule for their own specific lifestyle. Whether its to sleep in and attend courses later in the day or to get all their courses out of the way as early in the day as possible, our project allows them to choose their preference. We did this by taking in data directly from the MyUW course guide, creating a schedulizer that found a schedule for a student taking our sample classes, one avoids conflicts and includes all necessary times for that class, and finally adding functions that use the classes start and end times in order to find a best fit for that student based on his or her preferences. 

One possible future direction would be to add a feature that would detect the location of where a certain class is and factor that in to the time between classes. The reason that is sometimes a problem is because the current UW Schedule Planner can give you a class that start 5 minutes after a previous class ends, without factoring in the 15 minute walk between buildings, and thus students end up being late to their second class or leaving early from their first class. Future work would include an optimizer taking into account the location for each class, calculating the distance between two consecutive classes, and constraining this distance to be walkable at an average human walking speed (about 3.1 miles per hour according to TranSafety, Inc) within the alotted time for those classes.